In [ ]:
import pandas as pd
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import numpy as np
from datetime import datetime, timedelta


In [ ]:
# Snowflake接続設定
conn_params = {
    'user': 'YOUR_USERNAME',
    'password': 'YOUR_PASSWORD',
    'account': 'YOUR_ACCOUNT',
    'warehouse': 'YOUR_WAREHOUSE',
    'database': 'YOUR_DATABASE',
    'schema': 'YOUR_SCHEMA'
}

# Snowflakeに接続
conn = snowflake.connector.connect(**conn_params)
cursor = conn.cursor()


In [ ]:
# パターン1: SnowflakeのRANDOM()関数を使った乱数生成
query1 = """
SELECT 
    ROW_NUMBER() OVER (ORDER BY RANDOM()) as id,
    RANDOM() as random_float,
    FLOOR(RANDOM() * 100) as random_int_0_100,
    RANDOM() * 1000 as random_float_scaled,
    UNIFORM(1, 100, RANDOM()) as uniform_int,
    NORMAL(50, 10, RANDOM()) as normal_distribution
FROM TABLE(GENERATOR(ROWCOUNT => 1000))
ORDER BY id;
"""

# SQLを実行してデータを取得
df_random = pd.read_sql(query1, conn)
print("乱数データ生成完了!")
print(df_random.head())


In [ ]:
# パターン2: より実用的な乱数データ生成（ダミーデータ作成）
query2 = """
SELECT 
    UUID_STRING() as user_id,
    'user_' || FLOOR(RANDOM() * 10000) as username,
    CASE 
        WHEN RANDOM() < 0.5 THEN 'Male' 
        ELSE 'Female' 
    END as gender,
    FLOOR(RANDOM() * 60 + 18) as age,
    DATEADD(
        day, 
        FLOOR(RANDOM() * 365 * 5), 
        CURRENT_DATE - INTERVAL '5 years'
    ) as registration_date,
    RANDOM() * 100000 as account_balance,
    CASE 
        WHEN RANDOM() < 0.3 THEN 'Premium'
        WHEN RANDOM() < 0.7 THEN 'Standard'
        ELSE 'Basic'
    END as membership_tier
FROM TABLE(GENERATOR(ROWCOUNT => 500))
ORDER BY RANDOM();
"""

# SQLを実行してダミーユーザーデータを取得
df_users = pd.read_sql(query2, conn)
print("ダミーユーザーデータ生成完了!")
print(df_users.head())


In [ ]:
# パターン3: 時系列ランダムデータ生成（売上データなど）
query3 = """
WITH date_series AS (
    SELECT DATEADD(day, ROW_NUMBER() OVER (ORDER BY RANDOM()) - 1, '2023-01-01') as date
    FROM TABLE(GENERATOR(ROWCOUNT => 365))
)
SELECT 
    date,
    'product_' || FLOOR(RANDOM() * 100) as product_id,
    FLOOR(RANDOM() * 50 + 1) as quantity,
    ROUND(RANDOM() * 500 + 10, 2) as unit_price,
    ROUND((RANDOM() * 50 + 1) * (RANDOM() * 500 + 10), 2) as total_amount,
    CASE 
        WHEN RANDOM() < 0.1 THEN 'Cancelled'
        WHEN RANDOM() < 0.05 THEN 'Returned'
        ELSE 'Completed'
    END as status
FROM date_series
ORDER BY date, RANDOM();
"""

# SQLを実行して時系列データを取得
df_sales = pd.read_sql(query3, conn)
print("時系列売上データ生成完了!")
print(df_sales.head())
print(f"データポイント数: {len(df_sales)}")


In [ ]:
# データの統計情報表示と可視化
print("=== 乱数データの統計情報 ===")
print(df_random.describe())

print("\n=== ユーザーデータの統計情報 ===")
print(df_users.describe())

print("\n=== 売上データの統計情報 ===")
print(df_sales.describe())

# 簡単な集計例
print("\n=== 年齢分布 ===")
print(df_users['AGE'].value_counts().head())

print("\n=== 会員ランク分布 ===")
print(df_users['MEMBERSHIP_TIER'].value_counts())

print("\n=== 月別売上合計 ===")
df_sales['MONTH'] = pd.to_datetime(df_sales['DATE']).dt.to_period('M')
monthly_sales = df_sales.groupby('MONTH')['TOTAL_AMOUNT'].sum()
print(monthly_sales.head())


In [ ]:
# 接続のクリーンアップ
cursor.close()
conn.close()
print("Snowflake接続を閉じました。")

# 使用方法のメモ
print("""
使用方法:
1. conn_paramsの接続情報を実際の値に変更してください
2. 各セルを順番に実行してください
3. 生成されたデータはdf_random, df_users, df_salesに格納されます

生成される乱数データ:
- df_random: 基本的な乱数（浮動小数点、整数、正規分布など）
- df_users: ダミーユーザーデータ（UUID、年齢、性別、会員ランクなど）  
- df_sales: 時系列売上データ（商品ID、数量、価格、ステータスなど）
""")


In [ ]:
print("久しぶりのPythonだぜ！！！！！")